# AWS CloudFormation Stack Lifecycle

## 클라우드포메이션의 스택
- 클라우드포메이션으로 리소스를 프로비전할 때 관련된 리소스를 묶은 단위
- 스택의 생명 주기
    - 생성 -> 업데이트 -> 삭제 (스택 내부 리소스를 지우거나 새로 생성하는 것은 업데이트. 스택 자체를 지우는 것이 삭제)
- Nested Stack : 하나의 스택 안에 다른 스택을 포함하여 생성하는 방식
- Stack Set : 한번의 동작으로 여러 게정과 리전에 스택을 프로비전할 수 있는 기능

## 스택의 생명 주기 - 생성
- 생성 : 클라우드포메이션 템플릿을 기반으로 생성
    - S3 경로를 지정/직접 템플릿 업로드(S3 업로드) or Git에서 동기화
- 가능한 설정
    - IAM 역할 : 별도로 지정한 IAM 역할을 사용해서 프로비전 가능
    - 실패 동작
        - 모든 리소스 롤백 : 모든 리소스 프로비전 or 전체 롤백, 하나라도 실패하면 전부 롤백
        - 성공한 리소스 보존 : 성공한 리소스는 그대로 두고 프로비전에 실패한 리소스만 롤백
    - 롤백 정책
        - 삭제 정책 활용 : DeletionPolicy대로 삭제 여부 판단
        - 모든 리소스 삭제 : 롤백 중 생성된 리소스는 무조건 삭제
    - 기타 : SNS 알림, 생성 제한시간 등

### 스택 생성 데모
1. 스택 생성(shouldfail.yaml)
    - 템플릿 소스에 S3 URL을 직접 입력 가능
    - 템플릿 파일 업로드를 사용하면, 실제로는 버킷에 템플릿이 업로드되는 방식
    - 첫 템플릿은 중복된 버킷 이름이 들어가서 생성 실패, 모든 리소스 삭제
2. 같은 스택을 생성해보자
    - 스택 실패 옵션 : 성공적으로 프로비저닝된 리소스 보존
    - 만들어진 리소스는 보존, 실패한 리소스는 삭제

## 스택 생명 주기 - 업데이트
- 기존 템플릿 기반으로 파라미터만 변경 or 새로운 템플릿 기반 업데이트
    - S3 경로를 지정, 직접 템플릿 업로드(S3 업로드) or Git 동기화
- 가능한 설정
    - 생성과 거의 동일
- Stack Policy : 스택 리소스의 불필요한 업데이트를 방지하기 위한 일종의 보호장치
    - 설정 시 명시적으로 허용한 리소스를 제외하고는 업데이트 불가능
    - 한번 설정 시 삭제 불가능
    - 주요 사용 사례 : 특정 리소스만 업데이트 허용, 특정 리소스의 삭제 방지(스택 삭제는 가능)
- 변경 세트로 변경 내용에 대해 미리 확인 가능
    - 단, 변경 성공 여부를 보장하지는 않음

### 스택 업데이트 데모
1. 스택 생성 (Original Resource)
    - 고급 옵션 : 스택 정책 파일 업로드(리소스 타입이 EC2인 리소스만 업데이트 허용하는 정책)
2. 새로운 템플릿으로 업데이트
    - Action Not Allowed By Stack Policy로 Fail 되는것 확인
    - EC2 인스턴스의 경우 정책 상으로는 업데이트 허용이지만, 스택 업데이트 옵션에서 모두 롤백을 선택했기 때문에 모조리 롤백
3. 새로움 템플릿으로 업데이트 : 성공적인 프로비전은 보존하도록
    - 인스턴스는 업데이트 성공, 나머지는 Fail 된다
4. 업데이트 시 템플릿 전송 전 마지막 단계에서 변경 사항을 확인할 수 있다

- Stack Policy는 콘솔에서 업데이트 불가능, CloudShell을 쓰자.

## 스택의 생명 주기 - 삭제
- 스택의 삭제 시 전체 리소스 삭제 시도
    - DeletionPolicy에 따라 삭제 되지 않는 리소스 지정 가능
- 삭제 불가능한 경우 (예: S3버킷에 파일이 남아있는 경우)
    - 다른 모든 리소스를 삭제 후 실패 상태(DELETE_FAILED)로 남김
    - 이후 강제 삭제 시도(다시 시도) 혹은 해당 리소스를 제외하고 스택 삭제 가능